In [1]:
#Load Pacakges
library('foreign')
library('dplyr')
library('lme4')
library('nlme')
library(lmerTest)
library(ltm)
library(performance)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix


Attaching package: ‘nlme’


The following object is masked from ‘package:lme4’:

    lmList


The following object is masked from ‘package:dplyr’:

    collapse



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: msm

Loading required package: polycor



In [2]:
#Load Data
dfLong = read.csv('../../Data/daily_dfLong.csv')
dx = read.spss('../../Data/DemosDx/Diagnosis_1.sav',to.data.frame = T)

Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“../../Data/DemosDx/Diagnosis_1.sav: Long string value labels record found (record type 7, subtype 21), but ignored”
Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“Undeclared level(s) 22 added in variable: eatnos_1”
Warning message in read.spss("../../Data/DemosDx/Diagnosis_1.sav", to.data.frame = T):
“Undeclared level(s) 10 added in variable: addhpr_1”


In [3]:
dx$comorbidity = 
dx[,5:124] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
rowSums()

In [4]:
#Find people with MDD as their principle diagnoses
dx_mddPrin = 
dx %>%
filter(grepl('Prin',mddsp_1)|grepl('Prin',mddsnp_1)|grepl('Prin', mddrp_1)|grepl('Prin',mddrnp_1))%>%
mutate(bpd = ifelse(grepl('Curr', bord_1), 1, 0))

In [5]:
#Find most prevalent cormorbid disorders
rank = 
dx_mddPrin[,5:124] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
colSums()

dx_rank = data.frame(names(rank),  rank)
dx_rank[order(dx_rank$rank,decreasing = T),]

,names.rank.,rank
,<fct>,<dbl>
mddrnp_1,mddrnp_1,2706
gad4_1,gad4_1,1603
ptsd4_1,ptsd4_1,999
bord_1,bord_1,672
socal4_1,socal4_1,670
mddsnp_1,mddsnp_1,361
dysth_1,dysth_1,352
pda_1,pda_1,339
panic_1,panic_1,299


In [6]:
#Merge daily data with dx
dfLongMDD = merge(dfLong, dx_mddPrin, by = 'ID1',all = F,no.dups = TRUE)

dfLongMDD = dfLongMDD[order(dfLongMDD$ID1, dfLongMDD$day),]
length(unique(dfLongMDD$ID)) #1667


[1] 1667

In [7]:
#First sessino info
first_session = 
dfLongMDD %>%
    group_by(ID1) %>%
    filter(row_number() == 1) %>%
    ungroup()

In [33]:
first_session$SCID_1

[1] no  yes no  no  no  no  no  yes no  no  no  no  no  no  no  no  no  no 
  [19] no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no 
  [37] no  no  no  no  no  no  no  no  no  no  no  no  no  no  yes no  no  no 
  [55] yes no  no  no  no  no  no  no  no  no  no  no  no  2   no  no  2   no 
  [73] yes no  yes no  no  no  no  yes no  no  no  no  no  no  no  no  no  no 
  [91] no  no  no  no  2   no  no  2   no  no  no  no  no  no  no  no  no  no 
 [109] no  yes no  no  no  no  no  no  no  no  no  no  no  no  no  no  2   yes
 [127] no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  yes no 
 [145] no  no  no  no  yes no  2   yes yes no  no  yes no  yes yes no  yes 2  
 [163] no  no  no  no  no  no  no  no  2   yes no  no  no  no  no  no  yes no 
 [181] no  no  no  yes no  no  no  yes no  yes no  no  no  no  no  no  no  no 
 [199] no  no  no  yes no  yes no  no  no  no  no  no  no  no  no  no  no  no 
 [217] no  no  no  no  2   no  no  no  no  2   no  no  no  no  no  no  no  no 
 [235] no  no  no  no  no  no  2   no  no  no  no  no  no  no  no  no  no  no 
 [253] no  no  no  no  no  no  no  yes no  no  2   no  no  no  yes yes no  no 
 [271] no  no  no  yes 2   no  no  no  no  no  no  yes no  no  no  no  no  no 
 [289] no  no  no  no  no  no  no  no  2   no  yes no  no  no  no  2   no  no 
 [307] no  no  no  no  yes no  no  2   no  no  no  yes no  no  no  no  no  2  
 [325] no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  2  
 [343] no  no  no  no  no  no  no  yes no  no  no  no  yes no  no  2   no  no 
 [361] 2   yes no  2   yes no  no  no  no  no  no  no  yes no  no  no  no  no 
 [379] no  no  no  no  no  no  no  no  no  no  no  2   no  no  no  no  no  no 
 [397] no  no  no  no  no  no  no  no  2   no  no  yes yes no  no  no  no  no 
 [415] no  no  yes no  no  yes no  2   no  no  yes yes no  no  yes no  no  no 
 [433] no  yes yes no  no  no  no  no  2   no  no  2   yes no  no  no  yes no 
 [451] yes yes 2   no  no  yes no  no  no  no  no  yes yes no  yes no  yes no 
 [469] no  yes yes 2   no  yes no  no  no  no  no  no  yes no  no  no  2   2  
 [487] no  no  2   no  no  2   no  no  no  no  no  no  no  2   no  2   no  no 
 [505] no  no  no  no  no  yes yes yes no  no  yes yes yes no  2   2   no  2  
 [523] no  2   no  no  no  2   no  no  no  no  2   no  no  no  no  no  yes yes
 [541] no  yes no  no  2   no  yes yes no  2   2   yes yes no  no  no  no  2  
 [559] no  no  no  yes no  no  no  no  no  no  no  no  no  yes no  2   no  no 
 [577] 2   no  no  no  no  2   no  no  no  yes no  no  no  no  no  no  yes no 
 [595] no  no  no  no  no  no  no  yes no  yes yes no  yes 2   no  no  no  yes
 [613] yes no  no  no  no  no  yes no  no  no  yes no  yes yes no  yes yes yes
 [631] no  2   yes no  no  no  no  no  no  no  yes yes yes yes no  no  no  no 
 [649] no  yes 2   no  no  yes no  no  no  no  no  no  no  no  2   no  no  no 
 [667] no  no  no  yes no  no  no  no  yes yes no  yes no  no  yes no  no  no 
 [685] no  no  no  no  no  no  no  no  no  2   no  no  no  no  no  no  no  no 
 [703] yes yes no  no  no  no  no  no  no  2   no  yes no  no  no  no  no  no 
 [721] 2   no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no  no 
 [739] no  no  no  yes yes no  2   2   no  no  yes no  no  no  no  no  yes yes
 [757] yes yes no  no  yes no  yes 2   no  2   no  yes no  no  no  no  yes no 
 [775] no  no  no  no  no  no  yes no  yes no  yes no  yes no  2   no  no  no 
 [793] no  no  no  no  no  yes no  no  no  no  no  no  no  no  no  no  no  no 
 [811] no  yes no  no  yes no  no  no  no  yes no  no  no  yes no  yes 2   no 
 [829] no  no  no  no  no  yes yes no  no  no  yes no  no  no  no  yes yes yes
 [847] no  no  yes no  no  yes yes no  no  no  no  no  no  yes no  2   yes no 
 [865] no  2   no  no  no  no  no  no  2   no  yes no  yes 2   no  no  yes yes
 [883] no  no  no  2   no  no  no  yes no  no  no  no  no  yes 2   no  no  no 
 [901] no  no  no  no  yes no  2   yes no  yes yes no

In [8]:
#Examine Treatment Completion Rate
table(first_session$DC_status_1) #Treatment Complete N = 1087 
table(first_session$DC_status_1)/nrow(first_session)*100 #65.21


                                                      0 
                                                      0 
                                                     11 
                                                     17 
                                                     12 
                                                      7 
Inappropriate (language)/Referred to another tx setting 
                                                     18 
                           Insurance no longer covering 
                                                     22 
                                          Nonattendance 
                                                    199 
                                                  Other 
                                                     30 
                                    Other_duplicated_10 
                                                     10 
                 Reported dissatisfaction, withdrew AMA 
                              


                                                      0 
                                               0.000000 
                                                     11 
                                               1.019796 
                                                     12 
                                               0.419916 
Inappropriate (language)/Referred to another tx setting 
                                               1.079784 
                           Insurance no longer covering 
                                               1.319736 
                                          Nonattendance 
                                              11.937612 
                                                  Other 
                                               1.799640 
                                    Other_duplicated_10 
                                               0.599880 
                 Reported dissatisfaction, withdrew AMA 
                              

In [9]:
#Treatment Completion Rate by BPD
table(first_session$DC_status_1, first_session$bpd)
table(ifelse(first_session$DC_status_1 == 'Treatment Complete','Complete','Incomplete'), first_session$bpd)
summary(glm(ifelse(first_session$DC_status_1 == 'Treatment Complete',1,0) ~first_session$bpd, family = 'binomial'))

                                                         
                                                            0   1
  0                                                         0   0
  11                                                        8   9
  12                                                        5   2
  Inappropriate (language)/Referred to another tx setting  13   5
  Insurance no longer covering                             17   5
  Nonattendance                                           144  55
  Other                                                    28   2
  Other_duplicated_10                                       6   4
  Reported dissatisfaction, withdrew AMA                   32   7
  Transferred to inpatient                                 39  30
  Treatment Complete                                      893 194
  Treatment nearly complete                                45  12
  Withdrew due to external limitations                     91  21

            
               0   1
  Complete   893 194
  Incomplete 428 152


Call:
glm(formula = ifelse(first_session$DC_status_1 == "Treatment Complete", 
    1, 0) ~ first_session$bpd, family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5013  -1.5013   0.8849   0.8849   1.0757  

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)        0.73546    0.05879  12.510  < 2e-16 ***
first_session$bpd -0.49149    0.12325  -3.988 6.67e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2154.3  on 1666  degrees of freedom
Residual deviance: 2138.6  on 1665  degrees of freedom
AIC: 2142.6

Number of Fisher Scoring iterations: 4


In [10]:
#Treatment Completion Rate by # of Comorbidities
summary(glm(ifelse(first_session$DC_status_1 == 'Treatment Complete',1,0) ~ first_session$comorbidity, family = 'binomial'))



Call:
glm(formula = ifelse(first_session$DC_status_1 == "Treatment Complete", 
    1, 0) ~ first_session$comorbidity, family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5252  -1.4287   0.9050   0.9251   1.0726  

Coefficients:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                0.78838    0.11301   6.976 3.03e-12 ***
first_session$comorbidity -0.05368    0.03351  -1.602    0.109    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2154.3  on 1666  degrees of freedom
Residual deviance: 2151.7  on 1665  degrees of freedom
AIC: 2155.7

Number of Fisher Scoring iterations: 4


In [11]:
completeSample = first_session[first_session$DC_status_1 == 'Treatment Complete',]
completeSample

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,adltadd_1,condct_1,other_1_1,other_1_dx_1,other_2_1,other_2_dx_1,other_3_1,other_3_dx_1,comorbidity,bpd
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>
6,0,3,3,4,1,1,2,3,3,⋯,Not present,Not present,Not present,,Not present,,Not present,,6,0
13,0,3,3,4,1,1,1,1,4,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
14,0,5,5,3,4,1,5,1,5,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
23,0,4,4,3,1,4,3,5,3,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,0
29,0,4,3,5,1,3,1,3,2,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0
32,0,5,5,3,1,5,1,1,4,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,1
42,0,5,5,3,2,5,2,3,4,⋯,Not present,Not present,Not present,,Not present,,Not present,,3,0
43,0,3,3,1,1,3,2,3,2,⋯,Not present,Not present,Not present,,Not present,,Not present,,1,0
44,0,4,5,5,5,5,1,4,1,⋯,Not present,Not present,Not present,,Not present,,Not present,,2,0


In [12]:
#How many people had comorbid BPD
table(completeSample$bpd)


  0   1 
893 194 

In [13]:
#Comorbidity rates
table(completeSample$comorbidity)
mean(completeSample$comorbidity) #2.92
sd(completeSample$comorbidity) #1.50


  0   1   2   3   4   5   6   7   8   9  10 
  7 160 323 270 181  79  40  19   4   3   1 

[1] 2.922723

[1] 1.49816

In [14]:
rank = 
completeSample[,142:261] %>%
dplyr::select(-contains('nos'))%>%
mutate_all(~ifelse(grepl('Curr',.),1, 0)) %>%
colSums()

dx_rank = data.frame(names(rank),  rank, rank/nrow(completeSample))
dx_rank[order(dx_rank$rank,decreasing = T),][1:10,]

,names.rank.,rank,rank.nrow.completeSample.
,<fct>,<dbl>,<dbl>
mddrnp_1,mddrnp_1,891,0.81968721
gad4_1,gad4_1,479,0.44066237
ptsd4_1,ptsd4_1,283,0.26034959
bord_1,bord_1,194,0.17847286
socal4_1,socal4_1,178,0.16375345
mddsnp_1,mddsnp_1,110,0.10119595
etohab_1,etohab_1,93,0.08555658
panic_1,panic_1,92,0.08463661
pda_1,pda_1,90,0.08279669


In [15]:
#Correlation between BPD and # of cormorbidities 
cor(completeSample$bpd,completeSample$comorbidity) 

[1] 0.3786272

In [16]:
#BPD and Comorbidities vs. Treatment Length
completeSample$tx_length = completeSample$Days_complete_1 + completeSample$Days_not_attended_1
completeSample %>%
group_by(bpd) %>%
summarize(mean = mean(tx_length), sd = sd(tx_length))

t.test(completeSample$tx_length[completeSample$bpd == 0],
      completeSample$tx_length[completeSample$bpd == 1])


completeSample$tx_length = completeSample$Days_complete_1 + completeSample$Days_not_attended_1
completeSample %>%
group_by(bpd) %>%
summarize(mean = mean(comorbidity), sd = sd(comorbidity))

t.test(completeSample$comorbidity[completeSample$bpd == 0],
      completeSample$comorbidity[completeSample$bpd == 1])


#completeSample$tx_attendence_rate = completeSample$Days_complete_1/completeSample$tx_length
#summary(lm(tx_length ~bpd + comorbidity, completeSample)) #Both BPD and Comorbidities predicted longer treatment length in tx completion sample
#summary(lm(tx_attendence_rate ~bpd + comorbidity , completeSample)) #BPD does not predicted attendence rate, while # of comorbidities predicted lower attendence rate

`summarise()` ungrouping output (override with `.groups` argument)



bpd,mean,sd
<dbl>,<dbl>,<dbl>
0,11.23068,5.168298
1,14.45876,11.007809



	Welch Two Sample t-test

data:  completeSample$tx_length[completeSample$bpd == 0] and completeSample$tx_length[completeSample$bpd == 1]
t = -3.9901, df = 211.82, p-value = 9.098e-05
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -4.822836 -1.633323
sample estimates:
mean of x mean of y 
 11.23068  14.45876 


`summarise()` ungrouping output (override with `.groups` argument)



bpd,mean,sd
<dbl>,<dbl>,<dbl>
0,2.658455,1.374344
1,4.139175,1.445454



	Welch Two Sample t-test

data:  completeSample$comorbidity[completeSample$bpd == 0] and completeSample$comorbidity[completeSample$bpd == 1]
t = -13.045, df = 273.97, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.704187 -1.257254
sample estimates:
mean of x mean of y 
 2.658455  4.139175 


In [17]:
#Inital Severity
completeSample %>% 
summarize(depM = mean(depMean, na.rm = T), depSD = sd(depMean, na.rm = T),
         anxM = mean(anxMean, na.rm = T), anxSD = sd(anxMean, na.rm = T),
         angM = mean(angMean, na.rm = T), angSD = sd(angMean, na.rm = T))

depM,depSD,anxM,anxSD,angM,angSD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.404691,0.7168148,3.1307,0.8878087,2.244165,0.9634773


In [18]:
table(completeSample$Gender_1)
table(completeSample$Gender_1)/nrow(completeSample)
nrow(completeSample)
0.003679853 + 0.005519779 


                                         Female 
                                            720 
                                           Male 
                                            347 
                                     Non-Binary 
                                              4 
                                          Other 
                                              6 
Unknown (ID as Trans Before Demos Change 12.18) 
                                              9 


                                         Female 
                                    0.662373505 
                                           Male 
                                    0.319227231 
                                     Non-Binary 
                                    0.003679853 
                                          Other 
                                    0.005519779 
Unknown (ID as Trans Before Demos Change 12.18) 
                                    0.008279669 

[1] 1087

[1] 0.009199632

In [37]:
completeSample$SCID_1[completeSample$SCID_1==2] = 'yes'

table(completeSample$SCID_1)

table(completeSample$SCID_1)/1087


  2  no yes 
  0 860 227 


        2        no       yes 
0.0000000 0.7911684 0.2088316 

In [19]:
mean(completeSample$Age_1)
sd(completeSample$Age_1)
range(completeSample$Age_1)

[1] 40.36523

[1] 14.25956

[1] 18 88

In [20]:
table(completeSample$Race_1)
table(completeSample$Race_1)/nrow(completeSample)



     Asian      Black   Hispanic      Other Portuguese      White 
        15         79        126         45         26        795 


     Asian      Black   Hispanic      Other Portuguese      White 
0.01379945 0.07267709 0.11591536 0.04139834 0.02391904 0.73137075 

### Complete Sample

In [83]:
dfLongMDDComplete = dfLongMDD[dfLongMDD$DC_status_1 == 'Treatment Complete',]
length(unique(dfLongMDDComplete$ID))
dfLongMDDComplete$txlength = dfLongMDDComplete$Days_complete_1 + dfLongMDDComplete$Days_not_attended_1

length(unique(dfLongMDDComplete[dfLongMDDComplete$bpd == 1, 'ID1']))

#ICC, Descriptive Test, t-Tests


model1 = lmer(depMean ~ day + (1|ID1), data =  dfLongMDDComplete) 
model2 = lmer(dep17 ~ day + (1|ID1), data =  dfLongMDDComplete) 
model3 = lmer(dep18 ~ day + (1|ID1), data =  dfLongMDDComplete) 
table1 = data.frame(Var = c('Sym', 'Fun', 'Well-being'),
    ICC = rbind(
    icc(model1, by_group = TRUE),
    icc(model2, by_group = TRUE),
    icc(model3, by_group = TRUE))[,2])

table1


[1] 1087

[1] 194

Var,ICC
<fct>,<dbl>
Sym,0.6702102
Fun,0.5810962
Well-being,0.5640719


In [84]:
meanSE = function(x){
    paste0(round(x$coefficient['bpd','Estimate'],2), ' (',
       round(x$coefficient['bpd','Std. Error'],2) ,')')
    
}

In [85]:

#Pre-Treatment Data
dfLongMDDComplete_Session1 = 
    dfLongMDDComplete %>%
    filter(day == 0)

#Pre-Treatment Depressive Symptoms
pre_dep_comp = summary(lm(depMean ~ bpd + SCID_1, dfLongMDDComplete_Session1))
pre_dep_comp

#Pre-Treatment Functionality
pre_fun_comp = summary(lm(dep17 ~ bpd + SCID_1, dfLongMDDComplete_Session1))
pre_fun_comp

#Pre-Treatment Well-Being
pre_well_comp = summary(lm(dep18 ~ bpd + SCID_1, dfLongMDDComplete_Session1))
pre_well_comp

table1 = cbind(table1, c(meanSE(pre_dep_comp), meanSE(pre_fun_comp), meanSE(pre_well_comp)))


#Comorbidity
summary(lm(comorbidity ~ bpd + SCID_1, dfLongMDDComplete_Session1))

#Treatment Length
summary(lm(txlength ~ bpd + SCID_1, dfLongMDDComplete_Session1))




Call:
lm(formula = depMean ~ bpd + SCID_1, data = dfLongMDDComplete_Session1)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.58655 -0.37970  0.06236  0.49530  1.65501 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.43764    0.09976  34.458  < 2e-16 ***
bpd          0.20686    0.06116   3.382 0.000749 ***
SCID_1no    -0.05794    0.10143  -0.571 0.567970    
SCID_1yes   -0.09265    0.11307  -0.819 0.412764    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.7085 on 945 degrees of freedom
  (114 observations deleted due to missingness)
Multiple R-squared:  0.01358,	Adjusted R-squared:  0.01045 
F-statistic: 4.338 on 3 and 945 DF,  p-value: 0.004787



Call:
lm(formula = dep17 ~ bpd + SCID_1, data = dfLongMDDComplete_Session1)

Residuals:
   Min     1Q Median     3Q    Max 
-2.926 -0.791  0.209  1.209  1.332 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   4.0838     0.1588  25.721   <2e-16 ***
bpd           0.1346     0.0984   1.368   0.1717    
SCID_1no     -0.2928     0.1615  -1.813   0.0701 .  
SCID_1yes    -0.4161     0.1801  -2.310   0.0211 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.127 on 932 degrees of freedom
  (127 observations deleted due to missingness)
Multiple R-squared:  0.008664,	Adjusted R-squared:  0.005473 
F-statistic: 2.715 on 3 and 932 DF,  p-value: 0.0437



Call:
lm(formula = dep18 ~ bpd + SCID_1, data = dfLongMDDComplete_Session1)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.6008 -0.6008  0.2873  0.3992  1.5179 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.94102    0.12262  32.140  < 2e-16 ***
bpd          0.11188    0.07621   1.468  0.14240    
SCID_1no    -0.34023    0.12472  -2.728  0.00649 ** 
SCID_1yes   -0.45894    0.13911  -3.299  0.00101 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8703 on 928 degrees of freedom
  (131 observations deleted due to missingness)
Multiple R-squared:  0.01535,	Adjusted R-squared:  0.01217 
F-statistic: 4.822 on 3 and 928 DF,  p-value: 0.002452



Call:
lm(formula = comorbidity ~ bpd + SCID_1, data = dfLongMDDComplete_Session1)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5247 -0.9051 -0.4339  0.5661  6.5661 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.52475    0.16598  21.236  < 2e-16 ***
bpd          1.47121    0.10613  13.862  < 2e-16 ***
SCID_1no    -1.09086    0.16927  -6.445 1.76e-10 ***
SCID_1yes    0.05354    0.19375   0.276    0.782    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.317 on 1059 degrees of freedom
Multiple R-squared:  0.2373,	Adjusted R-squared:  0.2352 
F-statistic: 109.8 on 3 and 1059 DF,  p-value: < 2.2e-16



Call:
lm(formula = txlength ~ bpd + SCID_1, data = dfLongMDDComplete_Session1)

Residuals:
    Min      1Q  Median      3Q     Max 
-11.508  -4.314  -1.431   2.569  94.492 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  12.1107     0.8330  14.539   <2e-16 ***
bpd           3.0769     0.5326   5.777    1e-08 ***
SCID_1no     -0.6793     0.8495  -0.800   0.4241    
SCID_1yes    -1.9903     0.9724  -2.047   0.0409 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 6.612 on 1059 degrees of freedom
Multiple R-squared:  0.0387,	Adjusted R-squared:  0.03598 
F-statistic: 14.21 on 3 and 1059 DF,  p-value: 4.379e-09


In [86]:
table1 = cbind(table1, pre = c(meanSE(pre_dep_comp), meanSE(pre_fun_comp), meanSE(pre_well_comp)))
table1

Var,ICC,pre
<fct>,<dbl>,<fct>
Sym,0.6702102,0.21 (0.06)
Fun,0.5810962,0.13 (0.1)
Well-being,0.5640719,0.11 (0.08)


In [87]:
#Post-Treatment Data
dfLongMDDComplete_SessionF = 
    dfLongMDDComplete %>%
    group_by(ID1) %>%
    slice(day = max(day))%>%
    ungroup() 

#Post-Treatment Depressive Symptoms
post_dep_comp = summary(lm(depMean ~ bpd + SCID_1, dfLongMDDComplete_SessionF))
post_dep_comp

#Post-Treatment Functionality
post_fun_comp = summary(lm(dep17 ~ bpd + SCID_1, dfLongMDDComplete_SessionF))
post_fun_comp

#Post-Treatment Well-Being
post_well_comp  = summary(lm(dep18 ~ bpd + SCID_1, dfLongMDDComplete_SessionF))
post_well_comp 

table1 = cbind(table1, post = c(meanSE(post_dep_comp), meanSE(post_fun_comp), meanSE(post_well_comp)))



Call:
lm(formula = depMean ~ bpd + SCID_1, data = dfLongMDDComplete_SessionF)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.4840 -0.6991 -0.0776  0.6080  3.6759 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  2.48404    0.13208  18.807   <2e-16 ***
bpd          0.06593    0.08909   0.740   0.4596    
SCID_1no    -0.15991    0.13510  -1.184   0.2370    
SCID_1yes   -0.31639    0.15559  -2.034   0.0424 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8557 on 650 degrees of freedom
  (270 observations deleted due to missingness)
Multiple R-squared:  0.008611,	Adjusted R-squared:  0.004036 
F-statistic: 1.882 on 3 and 650 DF,  p-value: 0.1313



Call:
lm(formula = dep17 ~ bpd + SCID_1, data = dfLongMDDComplete_SessionF)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.0581 -0.7892  0.1267  1.0763  2.3334 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.9739     0.1806  16.471   <2e-16 ***
bpd           0.0842     0.1207   0.697    0.486    
SCID_1no     -0.1848     0.1847  -1.000    0.318    
SCID_1yes    -0.3074     0.2120  -1.450    0.148    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.145 on 631 degrees of freedom
  (289 observations deleted due to missingness)
Multiple R-squared:  0.004599,	Adjusted R-squared:  -0.0001333 
F-statistic: 0.9718 on 3 and 631 DF,  p-value: 0.4055



Call:
lm(formula = dep18 ~ bpd + SCID_1, data = dfLongMDDComplete_SessionF)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.2203 -0.8886  0.1114  0.1431  2.1431 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  3.22030    0.14037  22.942   <2e-16 ***
bpd          0.10909    0.09377   1.163   0.2451    
SCID_1no    -0.33172    0.14350  -2.312   0.0211 *  
SCID_1yes   -0.36339    0.16497  -2.203   0.0280 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8984 on 637 degrees of freedom
  (283 observations deleted due to missingness)
Multiple R-squared:  0.01201,	Adjusted R-squared:  0.007359 
F-statistic: 2.582 on 3 and 637 DF,  p-value: 0.05254


In [88]:
#Change Data
dfLongMDDComplete_Change = 
    dfLongMDDComplete %>%
    group_by(ID1) %>%
    summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
    dep17C = dep17[n()]-dep17[1],
    dep18C = dep18[n()]-dep18[1],
    SCID_1 = SCID_1[1]) %>%
    ungroup()

#Post-Treatment Depressive Symptoms
change_dep_comp = summary(lm(depMeanC ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_dep_comp

#Post-Treatment Functionality
change_fun_comp = summary(lm(dep17C ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_fun_comp

#Post-Treatment Well-Being
change_well_comp = summary(lm(dep18C ~ bpd + SCID_1, dfLongMDDComplete_Change))
change_well_comp

table1 = cbind(table1, change = c(meanSE(change_dep_comp), meanSE(change_fun_comp), meanSE(change_well_comp)))



`summarise()` ungrouping output (override with `.groups` argument)




Call:
lm(formula = depMeanC ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.22132 -0.60178 -0.01085  0.65368  2.97961 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.16711    0.13938  -8.374 3.41e-16 ***
bpd         -0.02721    0.08649  -0.315    0.753    
SCID_1no     0.01342    0.14200   0.095    0.925    
SCID_1yes   -0.13455    0.16238  -0.829    0.408    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8516 on 652 degrees of freedom
  (431 observations deleted due to missingness)
Multiple R-squared:  0.00386,	Adjusted R-squared:  -0.0007235 
F-statistic: 0.8422 on 3 and 652 DF,  p-value: 0.4711



Call:
lm(formula = dep17C ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9153 -0.8685  0.1315  1.1315  4.2688 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.3949     0.2221  -6.280 6.32e-10 ***
bpd          -0.1373     0.1346  -1.020    0.308    
SCID_1no      0.2633     0.2257   1.167    0.244    
SCID_1yes     0.3102     0.2556   1.213    0.225    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.3 on 628 degrees of freedom
  (455 observations deleted due to missingness)
Multiple R-squared:  0.004668,	Adjusted R-squared:  -8.718e-05 
F-statistic: 0.9817 on 3 and 628 DF,  p-value: 0.4009



Call:
lm(formula = dep18C ~ bpd + SCID_1, data = dfLongMDDComplete_Change)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.2316 -0.2316 -0.1562  0.7684  4.7684 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.1394     0.1781  -6.398  3.1e-10 ***
bpd          -0.0754     0.1087  -0.693   0.4883    
SCID_1no      0.3710     0.1810   2.050   0.0408 *  
SCID_1yes     0.2955     0.2052   1.440   0.1503    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.042 on 623 degrees of freedom
  (460 observations deleted due to missingness)
Multiple R-squared:  0.008303,	Adjusted R-squared:  0.003527 
F-statistic: 1.739 on 3 and 623 DF,  p-value: 0.1578


In [89]:
table1

Var,ICC,pre,post,change
<fct>,<dbl>,<fct>,<fct>,<fct>
Sym,0.6702102,0.21 (0.06),0.07 (0.09),-0.03 (0.09)
Fun,0.5810962,0.13 (0.1),0.08 (0.12),-0.14 (0.13)
Well-being,0.5640719,0.11 (0.08),0.11 (0.09),-0.08 (0.11)


In [24]:
mean_sdc = 
dfLongMDDComplete %>%
group_by(ID1) %>%
summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
         dep17C = dep17[n()]-dep17[1],
         dep18c = dep18[n()]-dep18[1]) %>%
ungroup()%>%
dplyr::select(-ID1)%>%
group_by(bpd) %>%
summarize_all(mean_sd) %>%
t()

table1 = cbind(table1, mean_sdc[2:4,])
table1



t_test_change_data = 
dfLongMDDComplete %>%
group_by(ID1) %>%
summarize(bpd = bpd[1],
    depMeanC = depMean[n()] - depMean[1],
         dep17C = dep17[n()]-dep17[1],
         dep18c = dep18[n()]-dep18[1]) %>%
ungroup()%>%
dplyr::select(-ID1)

for (i in 1:3){
    print(t.test(t_test_change_data[t_test_change_data$bpd == 0,1+i],
    t_test_change_data[t_test_change_data$bpd == 1,1+i]))
}



`summarise()` ungrouping output (override with `.groups` argument)



,Var,ICC,1,2,1,2,1,2
,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
depMean,Sym,0.6702102,3.38 ( 0.7 ),3.59 ( 0.73 ),2.31 ( 0.85 ),2.39 ( 0.89 ),-1.18 ( 0.82 ),-1.2 ( 0.99 )
dep17,Fun,0.5810962,3.78 ( 1.14 ),3.94 ( 1.07 ),2.78 ( 1.13 ),2.88 ( 1.2 ),-1.14 ( 1.28 ),-1.29 ( 1.38 )
dep18,Well-being,0.5640719,3.59 ( 0.89 ),3.74 ( 0.8 ),2.9 ( 0.89 ),3.04 ( 0.96 ),-0.8 ( 1.04 ),-0.89 ( 1.05 )


`summarise()` ungrouping output (override with `.groups` argument)




	Welch Two Sample t-test

data:  t_test_change_data[t_test_change_data$bpd == 0, 1 + i] and t_test_change_data[t_test_change_data$bpd == 1, 1 + i]
t = 0.21567, df = 157.8, p-value = 0.8295
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.1701198  0.2118263
sample estimates:
mean of x mean of y 
-1.177525 -1.198378 


	Welch Two Sample t-test

data:  t_test_change_data[t_test_change_data$bpd == 0, 1 + i] and t_test_change_data[t_test_change_data$bpd == 1, 1 + i]
t = 1.1266, df = 162.81, p-value = 0.2616
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.1185103  0.4333994
sample estimates:
mean of x mean of y 
-1.135659 -1.293103 


	Welch Two Sample t-test

data:  t_test_change_data[t_test_change_data$bpd == 0, 1 + i] and t_test_change_data[t_test_change_data$bpd == 1, 1 + i]
t = 0.89712, df = 166.15, p-value = 0.371
alternative hypothesis: true difference in means is not equal to

In [25]:
#Predcit changes in depression with bpd while controling for txlength
print('Symptom reduction')
sum_sym = summary(lmer(depMean ~ day*bpd + (1+day| ID1), data = dfLongMDDComplete)) 
sum_sym$coefficients

print('Functionoing')
sum_fun = summary(lmer(dep17 ~ day*bpd + (1+day| ID1), data = dfLongMDDComplete))
sum_fun$coefficients

print('Wellbeing')
sum_well = summary(lmer(dep18 ~ day*bpd +   (1+day| ID1), data = dfLongMDDComplete))
sum_well$coefficients

[1] "Symptom reduction"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.07511357,0.024540032,1020.3417,125.310086,0.000000e+00
day,-0.11451390,0.003835257,628.7099,-29.858208,1.192299e-122
bpd,0.25004494,0.059145110,991.9500,4.227652,2.579453e-05
day:bpd,0.01893208,0.008980405,556.3372,2.108155,3.546419e-02


[1] "Functionoing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.56282374,0.035065679,1009.8757,101.604298,0.000000e+00
day,-0.10150232,0.004663468,580.8607,-21.765416,2.932622e-77
bpd,0.19591217,0.084190880,972.8106,2.327000,2.016977e-02
day:bpd,0.01605806,0.010604361,475.3005,1.514288,1.306175e-01


[1] "Wellbeing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.45545714,0.026409371,1016.3806,130.842082,0.000000e+00
day,-0.06800142,0.003470658,548.6479,-19.593236,3.359761e-65
bpd,0.16070578,0.063340496,974.5428,2.537173,1.133013e-02
day:bpd,0.01126395,0.007845611,440.0637,1.435701,1.517979e-01


In [26]:
#Chronbach's Alpha
cronbach.alpha(dfLongMDDComplete[dfLongMDDComplete$day == 0,grep('^dep[0-9]*$', colnames(dfLongMDDComplete))], na.rm = T)



Cronbach's alpha for the 'dfLongMDDComplete[dfLongMDDComplete$day == 0, grep("^dep[0-9]*$", ' '    colnames(dfLongMDDComplete))]' data-set

Items: 18
Sample units: 1063
alpha: 0.855


In [27]:
#Predcit changes in depression with bpd and comorbidities while controling for txlength

print('Symptom reduction')
sum_dep_co = summary(lmer(depMean ~ day*(bpd+comorbidity) +  (1+day| ID1), data = dfLongMDDComplete)) 
sum_dep_co$coefficients

print('Functionoing')
sum_fun_co = summary(lmer(dep17 ~ day*(bpd+comorbidity) +  (1+day| ID1), data = dfLongMDDComplete))
sum_fun_co$coefficients

print('Wellbeing')
sum_well_co = summary(lmer(dep18 ~ day*(bpd+comorbidity) +   (1+day| ID1), data = dfLongMDDComplete))
sum_well_co$coefficients

[1] "Symptom reduction"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),2.865287171,0.049114334,1022.9923,58.3391225,0.000000e+00
day,-0.135410397,0.007731774,629.9646,-17.5134966,3.009401e-56
bpd,0.134039601,0.063006045,996.1319,2.1274086,3.363128e-02
comorbidity,0.078599122,0.015950752,1013.2146,4.9276124,9.718160e-07
day:bpd,0.007591224,0.009610878,555.0635,0.7898575,4.299483e-01
day:comorbidity,0.007738217,0.002450587,589.8061,3.1576995,1.671437e-03


[1] "Functionoing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.241696989,0.070069025,1012.6200,46.2643368,5.383777e-252
day,-0.108641457,0.009463033,581.4677,-11.4806170,1.219521e-27
bpd,0.018521969,0.089525001,977.5121,0.2068916,8.361376e-01
comorbidity,0.120224771,0.022713707,998.3418,5.2930493,1.478993e-07
day:bpd,0.012541338,0.011406221,476.8030,1.0995174,2.720973e-01
day:comorbidity,0.002551908,0.002954899,528.2891,0.8636192,3.881890e-01


[1] "Wellbeing"


,Estimate,Std. Error,df,t value,Pr(>|t|)
(Intercept),3.300362559,0.053162585,1018.1924,62.080551,0.000000e+00
day,-0.082698714,0.007024756,546.9238,-11.772467,1.129725e-28
bpd,0.074692419,0.067863635,979.4454,1.100625,2.713303e-01
comorbidity,0.058263226,0.017226504,1002.1933,3.382185,7.469147e-04
day:bpd,0.003689842,0.008411469,441.2925,0.438668,6.611168e-01
day:comorbidity,0.005327018,0.002187008,492.3935,2.435756,1.521442e-02


In [28]:
#Interaction between bpd and txlength
print('Symptom reduction')
summary(lmer(depMean ~ (day*txlength)* bpd +  (1+day| ID1), data = dfLongMDDComplete)) 

#print('Functionoing')
#summary(lmer(dep17 ~ day*(bpd+comorbidity) + txlength +  (1+day| ID1), data = dfLongMDDComplete))

#print('Wellbeing')
#summary(lmer(dep18 ~ day*(bpd+comorbidity) + txlength +  (1+day| ID1), data = dfLongMDDComplete))

[1] "Symptom reduction"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ (day * txlength) * bpd + (1 + day | ID1)
   Data: dfLongMDDComplete

REML criterion at convergence: 14774.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9508 -0.5532 -0.0280  0.4913  7.2047 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.414966 0.64418       
          day         0.005618 0.07495  -0.17
 Residual             0.178209 0.42215       
Number of obs: 9366, groups:  ID1, 1021

Fixed effects:
                   Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)       2.752e+00  5.760e-02  1.017e+03  47.775  < 2e-16 ***
day              -2.390e-01  8.826e-03  7.784e+02 -27.080  < 2e-16 ***
txlength          3.068e-02  4.591e-03  9.550e+02   6.683 3.97e-11 ***
bpd               4.310e-01  1.029e-01  9.824e+02   4.190 3.04e-05 ***
day:txlength      1.011e-02  6.322e-04  5.560e+02  15.988  < 2e-16 ***
da

### Incomplete Sample

In [29]:
#Predcit changes in depression with bpd while controling for txlength

dfLongMDD$txlength = dfLongMDD$Days_complete_1 + dfLongMDD$Days_not_attended_1
dfLongMDD$completion = ifelse(dfLongMDD$DC_status_1 == 'Treatment Complete', 1, 0)

print('Symptom reduction')
summary(lmer(depMean ~ day*bpd *(completion + txlength) +  (1+day| ID1), data = dfLongMDD)) 

print('Functionoing')
summary(lmer(dep17 ~ day*bpd *(completion + txlength) +(1+day| ID1), data = dfLongMDD))

print('Wellbeing')
summary(lmer(dep18 ~ day*bpd *(completion + txlength) +(1+day| ID1), data = dfLongMDD))

[1] "Symptom reduction"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: depMean ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 18975.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9249 -0.5414 -0.0174  0.4884  7.1612 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.42693  0.6534        
          day         0.00573  0.0757   -0.13
 Residual             0.18058  0.4249        
Number of obs: 11724, groups:  ID1, 1522

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         2.999e+00  4.979e-02  1.626e+03  60.226  < 2e-16 ***
day                -2.143e-01  1.040e-02  1.232e+03 -20.598  < 2e-16 ***
bpd                 3.892e-01  9.051e-02  1.572e+03   4.300 1.82e-05 ***
completion         -1.948e-01  4.651e-02  1.580e+03  -4.188 2.97e-05 ***
txlength            2.589e-02  3.858e-03  1.403e+03   6.7

[1] "Functionoing"


Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00788615 (tol = 0.002, component 1)”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep17 ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 29085.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9999 -0.5636  0.0115  0.5441  4.2598 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.80878  0.89932       
          day         0.00821  0.09061  -0.33
 Residual             0.49274  0.70196       
Number of obs: 11480, groups:  ID1, 1516

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         3.325e+00  7.137e-02  1.632e+03  46.584  < 2e-16 ***
day                -1.886e-01  1.429e-02  1.119e+03 -13.200  < 2e-16 ***
bpd                 4.793e-01  1.290e-01  1.559e+03   3.716 0.000209 ***
completion         -2.694e-01  6.646e-02  1.579e+03  -4.053 5.30e-05 ***
txlength            4.803e-02  5.455e-03  1.348e+03   8.805

[1] "Wellbeing"


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dep18 ~ day * bpd * (completion + txlength) + (1 + day | ID1)
   Data: dfLongMDD

REML criterion at convergence: 23693.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0097 -0.5459 -0.0074  0.5262  4.4242 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ID1      (Intercept) 0.456953 0.67598       
          day         0.004198 0.06479  -0.29
 Residual             0.316486 0.56257       
Number of obs: 11453, groups:  ID1, 1516

Fixed effects:
                     Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)         3.275e+00  5.444e-02  1.636e+03  60.152  < 2e-16 ***
day                -1.306e-01  1.087e-02  1.058e+03 -12.015  < 2e-16 ***
bpd                 3.449e-01  9.834e-02  1.560e+03   3.507 0.000465 ***
completion         -2.338e-01  5.064e-02  1.581e+03  -4.616 4.22e-06 ***
txlength            3.928e-02  4.140e-03  1.327e+03   9.487